In [1]:
# import dependencies
from bs4 import BeautifulSoup as soup
from splinter import Browser
import pandas as pd
import requests
import numpy as np
import sqlite3
import os
import datetime
import time

# Scraper Code

### Single Scrape (dev code)

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# define the url and visit it with browser
storage_url = "http://www.gasnom.com/ip/freebird/oauc.cfm?type=1"
browser.visit(storage_url)

# grab the table data
tables = pd.read_html(storage_url)
storage_info = tables[0]
storage_raw = tables[2]

# reformat the table data
tsp_name = storage_info.iloc[0][1]
posting_date = storage_info.iloc[2][1]

storage_raw.columns = ["location_name","loc","loc_purp_desc","loc_qti","flow_ind","all_qty_avail",\
                       "design_capacity", "operating_capacity","tsq","oac","it_indicator","qty_reason","blanks"]
storage_raw = storage_raw.fillna(0)
storage_cl1 = storage_raw.loc[storage_raw['location_name'] != 0]
storage_cl1 = storage_cl1.drop(storage_cl1.index[0])
storage_cl1 = storage_cl1.reset_index(drop=True)
storage_cl2 = storage_cl1.iloc[:,[1,8]]

storage_cl2.insert(0, 'date', posting_date)
storage_cl2.insert(1, 'tsp_name', tsp_name)
storage_cl2

,date,tsp_name,loc,tsq
0,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,0002,0
1,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,021039,4100
2,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,012709,-649


### Scrape Loop

#### Scrape Today's Data

In [3]:
storage_names = ['caledonia', 'freebird', 'baygas','eastcheyenne']
column_names =['date','tsp_name',"loc","tsq"]

storage_df = pd.DataFrame(columns=column_names)

base_url = "http://www.gasnom.com/ip/"
end_url = "/oauc.cfm?type=1"

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

for name in storage_names :
    # define the url and visit it with browser
    storage_url = base_url + name + end_url
    browser.visit(storage_url)
    
    # grab the table data
    tables = pd.read_html(storage_url)
    storage_info = tables[0]
    storage_raw = tables[2]

    # reformat the table data
    tsp_name = storage_info.iloc[0][1]
    posting_date = storage_info.iloc[2][1]

    storage_raw.columns = ["location_name","loc","loc_purp_desc","loc_qti","flow_ind","all_qty_avail",\
                           "design_capacity", "operating_capacity","tsq","oac","it_indicator","qty_reason","blanks"]
    storage_raw = storage_raw.fillna(0)
    storage_cl1 = storage_raw.loc[storage_raw['location_name'] != 0]
    storage_cl1 = storage_cl1.drop(storage_cl1.index[0])
    storage_cl1 = storage_cl1.reset_index(drop=True)
    storage_cl2 = storage_cl1.iloc[:,[1,8]]

    storage_cl2.insert(0, 'date', posting_date)
    storage_cl2.insert(1, 'tsp_name', tsp_name)
    storage_df = storage_df.append(storage_cl2)

browser.quit()
storage_df

,date,tsp_name,loc,tsq
0,"October 10, 2020 09:05 PM CT","Caledonia Energy Partners, L.L.C.",542519,-2420
1,"October 10, 2020 09:05 PM CT","Caledonia Energy Partners, L.L.C.",542524,4950
0,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,0002,0
1,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,021039,4100
2,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,012709,-649
0,"October 10, 2020 09:05 PM CT",BayGas Storage,BG-1002,-120961
1,"October 10, 2020 09:05 PM CT",BayGas Storage,BG-1003,7450
2,"October 10, 2020 09:05 PM CT",BayGas Storage,BG-1008,0
3,"October 10, 2020 09:05 PM CT",BayGas Storage,BG-1013,149670
0,"October 10, 2020 09:05 PM CT",East Cheyenne Gas Storage,029734466,7424


#### Scrape Historic Data

In [ ]:
column_names =['date','tsp_name',"loc","tsq"]
storage_df = pd.DataFrame(columns=column_names)

storage_names = ['caledonia', 'freebird', 'baygas','eastcheyenne']
start = datetime.datetime(2017,9,30)
dates = [(start - datetime.timedelta(days=x)).strftime('%m/%d/%Y') for x in range(-1104, 0)]

base_url = "http://www.gasnom.com/ip/"
mid_url = "/oauc.cfm?dt="
end_url = "&type=1"

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

for name in storage_names :
    for date in dates :
    # define the url and visit it with browser
        storage_url = base_url + name + mid_url + date + end_url
        browser.visit(storage_url)

        # grab the table data
        tables = pd.read_html(storage_url)
        storage_info = tables[0]
        storage_raw = tables[2]

        # reformat the table data
        tsp_name = storage_info.iloc[0][1]
        posting_date = storage_info.iloc[2][1]

        storage_raw.columns = ["location_name","loc","loc_purp_desc","loc_qti","flow_ind","all_qty_avail",\
                               "design_capacity", "operating_capacity","tsq","oac","it_indicator","qty_reason","blanks"]
        storage_raw = storage_raw.fillna(0)
        storage_cl1 = storage_raw.loc[storage_raw['location_name'] != 0]
        storage_cl1 = storage_cl1.drop(storage_cl1.index[0])
        storage_cl1 = storage_cl1.reset_index(drop=True)
        storage_cl2 = storage_cl1.iloc[:,[1,8]]

        storage_cl2.insert(0, 'date', posting_date)
        storage_cl2.insert(1, 'tsp_name', tsp_name)
        storage_df = storage_df.append(storage_cl2)
        
        time.sleep(3) # Sleep for 3 seconds

browser.quit()
storage_df
        

# Database Code

In [ ]:
conn = sqlite3.connect('scraped_storage_data.sqlite')
c = conn.cursor()

In [ ]:
df.to_sql('storageData', conn, if_exists='append', index = False)

In [ ]:
c.execute("SELECT * FROM storageData")

rows = c.fetchall()

for row in rows:
    print(row)